---
title: Quantum-safe cryptography
description: In this lesson we will look at how moving to quantum-safe cryptography can protect data today against the threat of quantum algorithms.
---


{/* cspell:ignore SIDH primefactor, cryptosystems, Buchmann, Dahmen, Nazario, Diffie, DTMs, cryptologists, Homomorphic, Cryptologists, Eliece, Merkle, Isogeny, Supersingular, Zeadally, NTRU, isogeny, Sendrier, Dilithium, SPHINCS, FIPS, Lenstra, Lovász, smallsetminus,Miklós, Ajtai, Lyubashevsky, Micciancio, Díaz, Rolim, Regev, leftarrow, ciphertext, adots, liboqs, Stehle, Schwabe, kems, Decapsulation,recepient, kemalg, decapsulation, Ciphertext, plaintexts, OAEP, trival, ciphertexts, Fujisaki, Okamoto, Eiichiro, Okamato, Tatsuaki, Crytography */}

# Quantum-safe cryptography

In this lesson we will look at how moving to quantum-safe cryptography can protect data today against the threat of quantum algorithms that could break it's protection in future .

By the end of the lesson we will have covered:
 - What quantum-safe cryptography is
 - Standardization efforts including NIST competitions
 - Mathematical basis of some new algorithms such as Lattices and Learning with Errors
 - Some Python code examples demonstrating how new mathematic challenges can be used for encryption
 - A look at some specific quantum-safe algorithms including those in the CRYSTALS suite
 - Hybrid cryptography

## Introduction to quantum-safe cryptography

Since quantum computers represent a [distinct and potentially more powerful paradigm for computing](https://pubs.aip.org/aip/jap/article/132/16/160902/2837574/The-future-of-quantum-computing-with) than the classical computers in use today, cryptographic security needs to be reassessed for a world where quantum computers may <DefinitionTooltip definition="The rapid or exponential growth, increase, or multiplication of something, typically in a biological, technological, or abstract context.">proliferate</DefinitionTooltip>, enabling new cryptographic attacks that would not be possible using classical computers.

These attacks may occur in future on data that is being transmitted or stored now known as *harvest now, decrypt later*, so it is not sufficient to wait until the required systems are available, but to make changes now.

The field of <DefinitionTooltip definition="Also known as Post Quantum or Quantum-Resistant. Cryptographic algorithms, protocols, or systems that are designed to withstand attacks from quantum computers. Even as quantum computing technology progresses, the security of digital communication and sensitive information needs to remains intact.">**quantum-safe cryptography**</DefinitionTooltip> (QSC) encompasses efforts to [identify and develop cryptographic schemes](https://doi.org/10.1007/978-3-540-88702-7) that can withstand attacks both from quantum and classical computing systems. This is also sometimes called **quantum-resistant**, or **post-quantum cryptography**.`

In earlier lessons, we discussed some of the potential security risks that the development of general-purpose quantum computers poses to a number of traditional cryptographic primitives such as symmetric key algorithms, cryptographic hash functions, and asymmetric key algorithms. Additionally, we mentioned cryptographically relevant quantum algorithms and the protocols they impact.

We see that the most significant impacts of quantum algorithms occur in the context of asymmetric key cryptography, where [Shor's algorithm](https://doi.org/10.1109/SFCS.1994.365700) offers a polynomial-time solution to the prime factoring and <DefinitionTooltip definition="Discrete Logarithm Problem. Finding the exponent to which a given number (the base) must be raised to obtain another number (the result), within a specified mathematical group. Solving this problem for certain groups is computationally difficult, forming the basis for several cryptographic schemes but Quantum Computers may have the potential to solve this problem in the future.">discrete logarithm</DefinitionTooltip> problems. Therefore, asymmetric cryptosystems based on factoring and discrete logarithms need to be replaced by new quantum-safe cryptography schemes.

This is in contrast to the symmetric key and cryptographic hashing protocols impacted by the Grover and [BHT](https://doi.org/10.1007/BFb0054319) algorithms, where the quantum speedups are not super-polynomial. Therefore, in this latter case, existing algorithms such as AES and SHA-256 can be fortified at least in the medium term by ensuring sufficiently long key and hash lengths.

## Quantum algorithms and impacts to cryptography

| Quantum Algorithm | Functionality      | <DefinitionTooltip definition="As defined by the [NIST](https://csrc.nist.gov/glossary/term/security_strength): *A number characterizing the amount of work that is expected to suffice to defeat an implemented cryptographic mechanism (for example, by compromising its functionality and/or circumventing the protection that its use was intended to facilitate). Security strength is often expressed in bits. If the security strength of a particular implementation of a cryptographic mechanism is s bits, it is expected that the equivalent of (roughly) 2<sup>s</sup> basic operations of some sort will be sufficient to defeat it in some way.*">Security Strength</DefinitionTooltip> ($n$ = number of bits) | Impacted Cryptographic Protocols                 | Mitigation             |
|-------------------|--------------------|------------------------------------------------------|--------------------------------------------------|------------------------|
| Shor              | factoring          | $poly(n)$                                            | RSA                                              | Migrate to QSC         |
| Shor              | discrete logarithm | $poly(n)$                                            | Diffie-Hellman, DSA, Elliptic Curve Cryptography | Migrate to QSC         |
| Grover            | key search         | $2^{n/2}$                                            | Symmetric key algorithms (for example, AES)             | Sufficient key length  |
| Grover            | pre-image attack   | $2^{n/2}$                                            | Hash functions (for example, SHA-256)                   | Sufficient hash length |
| BHT               | collision attack   | $2^{n/3}$ or $2^{2n/5}$                              | Hash functions (for example, SHA-256)                   | Sufficient hash length |

## Basic principles of quantum-safe cryptography

### Computational complexity

In cryptography, the *computational complexity class* [**NP (non-deterministic polynomial time)**](https://en.wikipedia.org/w/index.php?title=NP_(complexity)&oldid=1158889203) plays an important role. This class consists of decision problems for which proposed solutions can be verified in polynomial time using a <DefinitionTooltip definition="Machine that operates on an infinite tape containing symbols which has a read/write head that can move left or right on the tape and change the symbols according to a set of predetermined rules. In a deterministic Turing machine, the transition from one state to another is uniquely determined by the current state and the symbol being read from the tape. This means that for any given configuration of the machine, there is only one possible next step.">deterministic Turing machine</DefinitionTooltip> (DTM). The importance of NP stems from the fact that it is conjectured to consist of many computational problems that cannot be solved efficiently by both classical and quantum computers.

The first generation of successful asymmetric key cryptosystems developed in the 1970s based their security on mathematical problems such as prime factorization and discrete logarithms that are now conjectured to belong to the **NP-intermediate** subclass of NP. This subclass consists of problems that are believed not to have polynomial-time solutions on DTMs but at the same time are also not as hard as the hardest problems in NP.

The latter belong to the subclass [**NP-complete**](https://en.wikipedia.org/w/index.php?title=NP-completeness&oldid=1157123934). Following [Shor's algorithm](https://doi.org/10.1109/SFCS.1994.365700) in the 1990s, it became clear that at least some NP-intermediate problems are amenable to efficient solutions on quantum computers that are not DTMs.

Therefore, modern quantum-safe cryptography schemes are based on NP-complete problems or related [**NP-hard**](https://en.wikipedia.org/w/index.php?title=NP-hardness&oldid=1153995836) problems, which currently are not known to be solvable efficiently even on quantum computers.

### Average vs worst-case hardness

While there are many known NP-hard problems, not every such problem is suitable as a basis for cryptographic security. In this context, the notion of [**average-case hardness**](https://en.wikipedia.org/w/index.php?title=Computational_hardness_assumption&oldid=1158033193) is useful for cryptography. A problem is **average-case hard** if most instances of the problem drawn randomly from some distribution are hard, whereas a problem is **worst-case hard** if it is hard only on some isolated *worst-case* instances. Quantum-safe cryptologists therefore search for mathematical problems that satisfy the assumption of average-case hardness and employ theoretical tools such as worst-case to average-case [reductions](http://arxiv.org/abs/2208.08125) to identify suitable protocols whose security and efficiency can be guaranteed.

### Mathematical structures

Cryptologists have put forth a number of different mathematical structures that satisfy the necessary hardness requirements as potential candidates for quantum-safe migration of asymmetric key cryptosystems. Some well-known families include:

1. [**Lattice-based cryptography:**](http://arxiv.org/abs/2208.08125) This class of algorithms relies on the hardness of problems such as the shortest vector problem (SVP) and the closest vector problem (CVP) in [lattice structures](https://en.wikipedia.org/w/index.php?title=Lattice_problem&oldid=1149662985). Notable lattice-based schemes include [NTRU](https://en.wikipedia.org/w/index.php?title=NTRU&oldid=1107481357) and [Learning with Errors](https://doi.org/10.1109/CCC.2010.26) (LWE).

2. [**Code-based cryptography:**](https://doi.org/10.1007/978-3-540-88702-7_4) This type of cryptography is based on the difficulty of decoding a general linear code. The most notable example is the [McEliece cryptosystem](https://en.wikipedia.org/w/index.php?title=McEliece_cryptosystem&oldid=1158412532).

3. [**Multivariate cryptography:**](https://en.wikipedia.org/w/index.php?title=Multivariate_cryptography&oldid=1117344303) These systems involve equations of multiple variables over a finite field. A well-known system in this category is the HFE (Hidden Field Equations) scheme.

4. [**Hash-based cryptography:**](https://en.wikipedia.org/w/index.php?title=Hash-based_cryptography&oldid=1159153101) These are cryptographic systems that use only cryptographic hash functions. They are often used for digital signatures, like the Merkle signature scheme.

5. [**Isogeny-based cryptography:**](https://www.computer.org/csdl/magazine/it/2019/06/08896171/1eS9UwZEbjG) These systems are based on the difficulty of certain problems in the algebraic structure of elliptic curves. [Supersingular Isogeny Diffie-Hellman](https://en.wikipedia.org/w/index.php?title=Supersingular_isogeny_key_exchange&oldid=1163291033) (SIDH) is an example.

## NIST standardization of quantum-safe cryptography

Recognizing the potential impact of quantum computing on current cryptographic systems, NIST initiated a program to standardize quantum-safe cryptographic algorithms in 2016 using a process similar to the one NIST followed to standardize the Advanced Encryption Standard (AES) in the early 2000s. In an open and transparent process involving security domain stakeholders, several QSC candidates were submitted for evaluation, and following a six-year review process, [NIST announced a list of four finalists](https://www.nist.gov/news-events/news/2022/07/nist-announces-first-four-quantum-resistant-cryptographic-algorithms) to become a part of NIST’s quantum-safe cryptographic standard.

### Finalists of the first NIST quantum-safe cryptography standardization effort

| QSC Algorithm      | Cryptographic family | Application                     |
|--------------------|----------------------|---------------------------------|
| CRYSTALS-Kyber     | Lattice-based        | Key encapsulation mechanism     |
| CRYSTALS-Dilithium | Lattice-based        | Digital signatures              |
| FALCON             | Lattice-based        | Lightweight digital signatures |
| SPHINCS+           | Hash-based           | Digital Signatures              |


Of the four NIST finalists, three are lattice-based and one, [SPHINCS+](https://sphincs.org/), is hash-based. The Kyber and Dilithium algorithms, part of the [CRYSTALS cryptographic suite](https://pq-crystals.org/), were selected to be general-purpose protocols for key encapsulation and digital signatures, respectively. [FALCON](https://en.wikipedia.org/w/index.php?title=Falcon_(signature_scheme)&oldid=1157852190) was recommended for applications requiring smaller digital signatures than those provided by Dilithium. [SPHINCS+](https://sphincs.org/) meanwhile was primarily chosen as a backup option to the lattice-based approaches, as it uses a different mathematical structure. Lattice-based cryptography therefore seems well-positioned to form the basis for the first-generation of QSC standards.

In August 2023, [NIST published three draft standards](https://csrc.nist.gov/news/2023/three-draft-fips-for-post-quantum-cryptography) for comments - which include the algorithms above:
- [FIPS 203, Module-Lattice-Based Key-Encapsulation Mechanism Standard](https://csrc.nist.gov/pubs/fips/203/ipd)
- [FIPS 204, Module-Lattice-Based Digital Signature Standard](https://csrc.nist.gov/pubs/fips/204/ipd)
- [FIPS 205, Stateless Hash-Based Digital Signature Standard](https://csrc.nist.gov/pubs/fips/205/ipd)

## Lattice-based cryptography

As the name suggests, [lattice-based cryptography](https://en.wikipedia.org/w/index.php?title=Lattice-based_cryptography&oldid=1163912863) (LBC) is based on the hardness of certain problems defined on mathematical structures called [**lattices**](https://en.wikipedia.org/w/index.php?title=Lattice_problem&oldid=1149662985).

Of fundamental importance are two computational problems on lattices, namely the **shortest vector problem** and the **learning with errors problem**, which we discuss below after some preliminary definitions.

### Lattices

We can think of what a lattice is in the world around us. The Eiffel tower (Paris), or Birds-nest stadium (Beijing). Those structures have many linked elements which can be thought of as a series of points joined together along straight lines.

Diamonds are another form of lattice. There is a clear, repeating structure of Carbon atoms. They are then effectively joined together by atomic bonds; and this structure that is formed is what gives diamond its unique properties. However, different crystal structures can have very different layouts (can be thought of as our lattice structure), and therefore ends up with distinct features. Hence, lattices serve as the foundational mathematical framework that dictates the unique traits of different structures which we use in cryptography.

In simple mathematical terms, a lattice can be thought of as a collection of regularly spaced points that repeat at fixed intervals. To describe how these points are positioned relative to each other, we use vectors. The specific arrangement of these vectors, which defines the layout of the lattice, is referred to as the *basis.*

Imagine you have a box of toy construction bricks, and you want to build various objects with the same set of pieces. Each unique object you create requires a specific arrangement, and the way you choose and arrange the bricks serves as the *basis* for constructing different objects. If you change the selection or arrangement of bricks, you get a different object with distinct characteristics.

Similarly, in a lattice, the *basis* is like the set of atomic building blocks that define the lattice's structure. Depending on how you arrange these building blocks (atoms or points) in the lattice, you can create various lattice structures with different properties. Just as changing the toy construction brick pieces changes the object you build, altering the basis changes the lattice's characteristics and properties.

It's important to note that lattices are not limited to two or three dimensions; they can extend to higher dimensions, and in fields like cryptography, they may involve 1000s or more dimensions.

<details>
<summary>Mathematical description of lattices</summary>

- Given $n$ linearly independent vectors $A = \{\mathbf{v_1},...,\mathbf{v_n} | \mathbf{v_i} \in \mathbb{R}^m\}$, the *lattice* $\mathcal{L}$ generated by $A$ is the set of integer linear combinations $\mathcal{L} = \{ \sum_i c_i \mathbf{v_i} | c_i \in \mathbb{Z} \}$
- A *basis* $B$ of a lattice $\mathcal{L}$ is any set of linearly independent vectors $B=\{b_1,...,b_n\}$  that spans the lattice i.e, $\mathcal{L}(B) = \{ \sum_i z_i b_i | z_i \in \mathbb{Z} \}$.
- Importantly, the basis for a given lattice is not unique. Two different basis sets $B, B^{\prime}$ can describe the same lattice.

</details>

Not every basis is unique - it may just be a different perspective of the same structure.

This leads to an important concept in lattice mathematics, that of [*lattice-basis reduction*](https://en.wikipedia.org/w/index.php?title=Lattice_reduction&oldid=1139588628). This is the process of taking a given integer lattice and attempting to find a *good* basis comprising short, nearly orthogonal vectors.

![Fig 1: Lattice-basis reduction in two dimensions from a "bad" basis v_1, v_2 to a good basis u_1, u_2](/learning/images/courses/quantum-safe-cryptography/quantum-safe-cryptography/Lattice-reduction-wiki.avif)

*Figure 1. Lattice-basis reduction in two dimensions from a "bad" basis $v_1$, $v_2$ to a good basis $u_1$, $u_2$*

Lattice-basis reductions can be performed in polynomial-time using the [**Lenstra–Lenstra–Lovász** algorithm](https://en.wikipedia.org/w/index.php?title=Lenstra%E2%80%93Lenstra%E2%80%93Lov%C3%A1sz_lattice_basis_reduction_algorithm&oldid=1151925724) (LLL).

### The shortest vector problem

The **shortest vector problem** (SVP) is the challenge of finding the shortest distance (vector), between any two points in the lattice

<details>
<summary>Mathematical description of the SVP</summary>

Given a lattice $\mathcal{L}$ embedded in a vector space $\mathcal{V}$ with norm $N$, the [SVP](https://en.wikipedia.org/w/index.php?title=Lattice_problem&oldid=1149662985) asks for the shortest non-zero vector in $\mathcal{L}$ as measured by $N$, that is, find a vector $v$ such that its length $\|v\|_N = \min\limits_{v^{\prime} \in \mathcal{L} \smallsetminus \{\mathbf{0}\}} \|v^{\prime}\|_N = \lambda(\mathcal{L})$.

</details>

![Fig 2: The SVP on a lattice specified by basis vectors b_1, b_2: The orange vector is the shortest vector](/learning/images/courses/quantum-safe-cryptography/quantum-safe-cryptography/SVP.avif)

*Figure 2. The SVP on a lattice specified by basis vectors $b_1$, $b_2$: The yellow vector is the shortest vector*

[Miklós Ajtai has shown that the shortest vector problem is **NP-hard**](https://dl.acm.org/doi/10.1145/237814.237838) in the worst case for certain random lattices, even when lattice-basis reduction to a *good* basis is possible.

Ajtai also demonstrated that there exists a worst-case to average-case reduction for the shortest vector problem, laying the foundation for its use in cryptography.

### The closest vector problem

In the **closest vector problem** (CVP) we are looking to find the closest point on the lattice to the origin, or location given.

The CVP is also known to be NP-hard.

![Fig 3: The CVP on a lattice specified by basis vectors b_1, b_2: The red point represents the closest lattice vector to the given external vector shown in green that is not on the lattice](/learning/images/courses/quantum-safe-cryptography/quantum-safe-cryptography/CVP.avif)

*Figure 3. The CVP on a lattice specified by basis vectors $b_1$, $b_2$: The red point represents the closest lattice vector to the given external vector shown in green that is not on the lattice*

The *shortest vector problem* is a special case of the *closest vector problem*.

<details>
<summary>Mathematical description of CVP</summary>

A lattice $\mathcal{L}$ embedded in a vector space $\mathcal{V}$ with metric $M$ is given, along with a vector $v$ in $\mathcal{V}$ but not necessarily in $\mathcal{L}$. The task then is to find the vector $u$ in $\mathcal{L}$ closest to $v$ as measured by $M$ that is, $d_M (u, v) = \min\limits_{u^{\prime} \in \mathcal{L}} d_M(u^{\prime}, v)$ where $d_M$ is the distance function of the metric $M$.

</details>

### The bounded distance decoding problem

Very similar to the CVP is the [**bounded distance decoding**](https://doi.org/10.1007/11830924_41) (BDD) problem.

In this case we know that the origin supplied is close to the lattice, but not *how close*, ie *bounded*.

<details>
<summary>Mathematical description of the BDD problem</summary>

It is additionally specified that the external vector $v$ is at most within a distance $\sigma\lambda(\mathcal{L})$, where $\lambda(\mathcal{L})$ is the length of the shortest vector in the lattice and $\sigma$ is a small parameter.

</details>

### The learning with errors problem


The learning with errors (LWE) problem is based on the mathematics of lattices combined with the idea of introducing errors into an equation.

As is the case with any mathematical basis for cryptographic algorithms we can easily introduce noise as we make a series of calculations, but trying to find the inverse is very difficult.

<details>
<summary>Mathematics of LWE problem</summary>

The [learning with errors problem](https://en.wikipedia.org/w/index.php?title=Learning_with_errors&amp;oldid=1153719613) ${\mathrm{LWE}_{q,\chi}}$ with modulus $q$ and error distribution $\chi$ is: Given access only to polynomially many samples $(\mathbf {a} ,t)$ of choice from Alice's distribution $A_{\mathbf{s},\chi }$, "learn" and output the secret vector ${\mathbf{s}}\in {\mathbb{Z}}_{q}^{n}$.

- Let ${\mathbb {Z} _{q}}$ be the ring of integers modulo $q$.
- Let ${\mathbb {Z}_{q}^{n}}$ be the set of $n$-dimensional vectors over ${\displaystyle \mathbb{Z}_{q}}$.
- An adversary — say, Alice — chooses a fixed vector ${\mathbf{s}} \in {\mathbb  {Z}}_{q}^{n}$ and keeps it a secret.
- Alice also specifies $\chi$, a fixed "error" probability distribution over $\mathbb {Z} _{q}$.
- Using the above ingredients, she constructs a distribution $A_{\mathbf{s},\chi }$ on ${\mathbb{Z}}_{q}^{n}\times{\mathbb {Z} _{q}}$ as follows:
    1. From the uniform distribution over ${\mathbb{Z}}_{q}^{n}$, Alice chooses a vector ${\mathbf{a}}\in {\mathbb{Z}}_{q}^{n}$ .
    2. From the given distribution $\chi$, she picks a number $ {\displaystyle \varepsilon \in \mathbb {Z} _{q} }$.
    3. She then evaluates ${\displaystyle t=\langle \mathbf{a} ,\mathbf {s} \rangle + \varepsilon}$, where $\langle\mathbf{a},\mathbf{s}\rangle =\sum_i a_{i}s_{i}$ is the standard inner product on ${\mathbb{Z}}_{q}^{n}$ and the addition is in $\mathbb {Z} _{q}$.
    4. Alice outputs ${\displaystyle (\mathbf {a} ,t)}$.

    Note the errors introduced: $\varepsilon$ drawn from $\chi$

    Early developments in the LWE problem employed the one-dimensional discrete Gaussian distribution on $\mathbb {Z} _{q}$ as the error distribution.

</details>

The remarkable feature of the LWE problem is that without the errors, it reduces to a set of linear equations that can be easily solved using a technique like Gaussian elimination.

However, the inclusion of a suitable error distribution renders it an NP-hard problem.

### Encryption using the Learning with Errors cryptosystem
The LWE problem discussed above leads to a public key cryptosystem introduced by [Regev](https://doi.org/10.1109/CCC.2010.26).

The exact process will be covered in the Python example below.

<details>
<summary>Mathematics of key generation, encryption, and decryption</summary>

**Parameters:** The cryptosystem is characterized by the vector dimension or security parameter $n$, modulus $q$, number of LWE samples $N$, and error distribution $\chi$. Typical choices that guarantee both security and correctness are:

- $q$ is a prime between $n^2$ and $2n^2$
- $N = 1.1\cdot~n~log~q$
- $\chi = \Psi_{\alpha}$, with $\alpha = 1/(\sqrt{n}~log^2{n})$, where $\Psi_{\alpha}$ is obtained by sampling the normal distribution $\mathcal{N}(0, \alpha^2/(2\pi))$ and reducing the result modulo 1

**Private key:** The private key is the secret vector ${\mathbf{s}} \in {\mathbb  {Z}}_{q}^{n}$.

**Public key:** Choose $N$ samples $(\mathbf{a}_i ,b_i)_{i=1}^N$ from the LWE distribution $A_{\mathbf{s},\chi }$

**Encryption:**

- Encryption utilizes the public key and is carried out bit by bit.
- For each bit of the message, randomly choose a binary vector $\mathbf{r} \in \{0,1\}^N$.
- If the message bit is 0, the encryption is $(\sum_i \mathbf{a}_i r_i, \sum_i b_i r_i)$.
- If the message bit is 1, the encryption is $(\sum_i \mathbf{a}_i r_i, \lfloor \frac{q}{2}\rfloor + \sum_i b_i r_i)$.

**Decryption:**

- The decryption utilizes the private key ${\mathbf{s}}$.
- The pair $({\mathbf{a}, b })$ decrypts to $0$ if $b-\langle \mathbf{a}\cdot\mathbf{s}\rangle$ is closer to $0$ than $\lfloor\frac{q}{2}\rfloor$ modulo $q$, else it decrypts to 1.


The security of the LWE public key cryptosystem was analyzed by Regev:

- Firstly, it was shown that recovering the error/noise vector $\mathbf{e} = [\varepsilon_1,...,\varepsilon_N] \leftarrow \chi^N$ involved in the construction of the public key is as hard as the bounded distance decoding (BDD) problem, which, as noted above, is believed to be NP-hard. This establishes the correspondence between LWE and explicit lattice problems.
- Secondly, security against chosen plain text attacks (CPA) is based on the observation that an algorithm for distinguishing between encryptions of 0 and 1 in the above cryptosystem can also distinguish between the distribution $A_{\mathbf{s},\chi }$ and the uniform distribution over $\mathbb{Z}_{q}^{n} \times \mathbb{Z}_{q}$, which would imply a solution of LWE itself.

</details>

### Illustration of LWE encryption in Python

The following simple example shows the use of <DefinitionTooltip definition="Learning With Errors. A mathematical technique which hides secret information by introducing errors into equations, making them harder to solve.">LWE</DefinitionTooltip> for encryption and decryption. Bob will send an encrypted message to Alice.

First, Alice and Bob agree on the **problem parameters**. These are explained in detail in the maths section above, but in summary we require $n, q, N, \chi$.

We start with some of the basic parameters

- $N$ represents the number of samples
- $q$ is a modulus
- n is known as the security parameter, or vector dimension.


These parameters are all public information in principle.

In [ ]:
import numpy as np

n = 8
q = 127
N = int(1.1 * n * np.log(q))
sigma = 1.0
print(f"n={n},q={q},N={N},sigma={sigma}")

We also need a way of introducing errors

Here $\chi$ represent the errors we want to introduce - we use a discrete Gaussian distribution on $\mathbb{Z}_{q}$ characterized by mean 0 and standard deviation $\sigma$.

This definition of the errors to introduce is also public.

So in Python, we need a simple function that approximates random error/noise contributions $\varepsilon$ drawn from a discrete Gaussian distribution $\chi$ with standard deviation $\sigma$.

Some example values are printed so that you can see the distribution around 0.

In [ ]:
def chi(stdev, modulus):
    return round((np.random.randn() * stdev**2)) % modulus


# print some examples
sd = 2
m = 1000
for x in range(10):
    print("chi = ", chi(sd, m))

Next, Alice needs to generate her key pair.

She sets up her **private key** by choosing $n$ values between 0 and $q$.

In [ ]:
# Alice's private key
alice_private_key = np.random.randint(0, high=q, size=n)
print(f"Alice's private key: {alice_private_key}")

Alice now sets up her **public key**, by choosing random vectors, which are then combined with the generated errors.

<details>
<summary>Mathematics</summary>

More precisely, she needs to generate the sample $A_{s,\chi}$.

Accordingly, she randomly chooses vectors $\mathbf{a} \in \mathbb{Z}_{q}^{n}$ and errors $\varepsilon$ from $\chi$ to construct $N$ samples $(\mathbf{a}, b)$.

</details>

In [ ]:
# Alice's Public Key
alice_public_key = []

# N is the number of values we want in the key
for i in range(N):
    # Get n random values between 0 and <q
    a = np.random.randint(0, high=q, size=n)
    # get an error to introduce
    epsilon = chi(sigma, q)
    #  calculate dot product (ie like array multiplication)
    b = (np.dot(a, alice_private_key) + epsilon) % q
    # value to be added to the key -
    sample = (a, b)
    alice_public_key.append(sample)

print(f"Alice's public key: {alice_public_key}")

Alice can now share her public key.

Bob can now use this to send Alice an encrypted message.

Bob's message is a single bit.

In [ ]:
# Encryption
bob_message_bit = 1
print(f"Bob's message bit={bob_message_bit}")

To encrypt the message, Bob needs to select an arbitrary number of samples at random from Alice's public key to form the ciphertext.

For this, he creates a mask, a random binary vector $r$ of length $N$.

In [ ]:
# a list of N values between 0 and <2 - ie 0 or 1
r = np.random.randint(0, 2, N)
print(r)

We now take this mask and apply it to the relevant entry in Alice's public key, calculating a sum of the values found.

In [ ]:
sum_ai = np.zeros(n, dtype=int)
sum_bi = 0

for i in range(N):
    sum_ai = sum_ai + r[i] * alice_public_key[i][0]
    sum_bi = sum_bi + r[i] * alice_public_key[i][1]
sum_ai = [x % q for x in sum_ai]
# sum_bi = sum_bi
ciphertext = (sum_ai, (bob_message_bit * int(np.floor(q / 2)) + sum_bi) % q)
print(f"ciphertext is: {ciphertext}")

Finally, Bob broadcasts the ciphertext, which Alice can decrypt using her private key.

In [ ]:
# Decryption
adots = np.dot(ciphertext[0], alice_private_key) % q
b_adots = (ciphertext[1] - adots) % q

decrypted_message_bit = round((2 * b_adots) / q) % 2

print(
    f"original message bit={bob_message_bit}, decrypted message bit={decrypted_message_bit}"
)

assert bob_message_bit == decrypted_message_bit

Since this protocol works bit by bit for encrypting longer bit strings, we simply repeat the operations in a loop. The following shows a scenario where Bob wishes to transfer 16 encrypted bits.

In [ ]:
bob_message_bits = np.random.randint(0, 2, 16)
print(f"Bob's message bits are : {bob_message_bits}")
decrypted_bits = []

for ib in range(len(bob_message_bits)):
    bob_message_bit = bob_message_bits[ib]

    r = np.random.randint(0, 2, N)

    sum_ai = np.zeros(n, dtype=int)
    sum_bi = 0
    for i in range(N):
        sum_ai = sum_ai + r[i] * alice_public_key[i][0]
        sum_bi = sum_bi + r[i] * alice_public_key[i][1]
    sum_ai = [x % q for x in sum_ai]

    ciphertext = (sum_ai, (bob_message_bit * int(np.floor(q / 2)) + sum_bi) % q)

    adots = np.dot(ciphertext[0], alice_private_key) % q
    b_adots = (ciphertext[1] - adots) % q

    decrypted_message_bit = round((2 * b_adots) / q) % 2
    assert decrypted_message_bit == bob_message_bit

    decrypted_bits.append(decrypted_message_bit)

print(f"Decrypted message bits = {np.array(decrypted_bits)}")

## Module-LWE and the CRYSTALS suite

The learning with errors (LWE) problem, introduced in a simplified form above and generally valid on arbitrary lattices, has been extended to algebraic *rings* within the so-called [**Ring-LWE**](https://en.wikipedia.org/w/index.php?title=Ring_learning_with_errors&oldid=1148731729) framework primarily to improve the efficiency of resulting cryptosystems. However, the extra algebraic structure of Ring-LWE may be potentially exploitable, even though no such exploits are currently known.

Two of the four finalists in NIST's QSC standardization process — namely, the [**CRYSTALS-Kyber**](https://pq-crystals.org/kyber/index.shtml) key encapsulation mechanism (KEM) and the [**CRYSTALS-Dilithium**](https://pq-crystals.org/dilithium/index.shtml) digital signature protocol — are based on structures known as <DefinitionTooltip definition="Collection of all submodules of a given module. A submodule is a subset of the module that itself forms a module under the same operations. The set of all submodules forms a lattice, which is a partially ordered set where any two elements have a unique least upper bound and greatest lower bound.">*module lattices*</DefinitionTooltip> and the related [**Module-LWE**](https://eprint.iacr.org/2012/090).

We refer interested readers to specialized courses on these topics for the mathematical details but note here that Module-LWE is seen as a compromise between LWE and Ring-LWE. It provides more efficiency than LWE (though less than Ring-LWE) but also gives a higher level of security assurance than Ring-LWE because it does not rely as heavily on the algebraic structure of the problem.

### Key Encapsulation Mechanisms and CRYSTALS-Kyber
Traditional asymmetric key cryptosystems are most heavily deployed for their *key-exchange* and *digital signature* functionalities and as such, the NIST standardization process sought to develop quantum-safe alternatives for these two functionalities. The [**CRYSTALS-Kyber**](https://pq-crystals.org/kyber/index.shtml) protocol is therefore designed as a dedicated [**Key Encapsulation Mechanism**](https://en.wikipedia.org/w/index.php?title=Key_encapsulation_mechanism&oldid=1110687965) (KEM) rather than as a general purpose encryption scheme such as RSA. A workflow illustrating the use of Kyber to establish a shared secret between two parties is shown below.

#### KEM example using liboqs (local setup required)

<Admonition type="note">
This example requires some additional local installation, and currently cannot be run on this site. You can review the output below, or alternatively copy the cells to a suitable local environment if you wish to run these examples**
</Admonition>

Here we make use of the algorithms provided by the `liboqs` open source project which provides a libraries for prototyping and experimenting with quantum-safe cryptography. Refer to the links below to setup and test, and then run these examples in your local Python environment.

- **Python Library:** This provides the Python language binding for liboqs - https://github.com/open-quantum-safe/liboqs-Python
- **liboqs implementation:** This is required by the Python library, and provides the actual algorithm implementations and must be built - https://github.com/open-quantum-safe/liboqs


Alice wishes to create and securely communicate a shared secret to Bob using a quantum-safe encryption approach. She will use the `Kyber512` KEM algorithm provided by liboqs.

We begin by importing relevant Python modules :

In [ ]:
import warnings
import oqs
from pprint import pprint

warnings.filterwarnings("ignore")

`liboqs` provides a number of KEM implementations and a list of the available algorithms can be easily obtained as follows

In [ ]:
kems = oqs.get_enabled_kem_mechanisms()
pprint(kems, compact=True)

We see that one of the options in the list is `Kyber512` which we will employ in this example. Key exchange using Kyber involves three simple steps namely
1. Key generation
2. Encapsulation
3. Decapsulation

In the key generation step, the recepient Bob, needs to generate an asymmetric key pair using `Kyber512` and broadcast his public key.

In [ ]:
kemalg = "Kyber512"
bob = oqs.KeyEncapsulation(kemalg)
bob_public_key = bob.generate_keypair()

Next in the **encapsulation** step, upon receiving Bob's public key, Alice generates both the shared secret and it's corresponding ciphertext using `Kyber512` and Bob's public key. The ciphertext which *encapsulates* the shared secret is then broadcasted to Bob.

In [ ]:
alice = oqs.KeyEncapsulation(kemalg)
ciphertext, shared_secret_alice = alice.encap_secret(bob_public_key)

Finally, in the **decapsulation** step, Bob uses his private key to recover the shared secret from the ciphertext.

In [ ]:
shared_secret_bob = bob.decap_secret(ciphertext)
print("\nShared secretes coincide:", shared_secret_bob == shared_secret_alice)

Note that in contrast to key exchange using RSA, no extraneous padding or key derivation functions are involved here. Kyber's design as a KEM allows for a very minimal user interface while providing the gold standard [**IND-CCA2**](https://en.wikipedia.org/w/index.php?title=Ciphertext_indistinguishability&oldid=1130147047) security for key exchange.

###  IND-CPA and IND-CCA security in lattice-based cryptography
In the lesson on Symmetric Key Cryptography, we introduced the notion of **semantic security** or **IND-CPA** security which refers to [**indistinguishability under chosen-plaintext attack**](https://en.wikipedia.org/w/index.php?title=Ciphertext_indistinguishability&oldid=1130147047).

Traditional cryptosystems whether symmetric (such as AES) or asymmetric (such as RSA) use deterministic functions to implement encryption operations. This means that a given plaintext, combined with a given encryption key will always encrypt to the same ciphertext. Such deterministic cryptosystems are vulnerable to *chosen plaintext attack* whereby an adversary is able to extract information by requesting encryptions of arbitrary plaintexts of their choice from the deterministic encryption function.

To achieve IND-CPA security in this context, **additional randomness** is introduced at encryption time either through *initialization vectors* or *padding*. For instance AES is only IND-CPA secure when used in [Cipher Block Chaining (CBC) or Gaolis/Counter Mode (GCM) modes of operation](https://en.wikipedia.org/w/index.php?title=Block_cipher_mode_of_operation&oldid=1154901199#cite_note-23) that use random initialization vectors. Similarly with RSA, [OAEP padding](https://en.wikipedia.org/w/index.php?title=Optimal_asymmetric_encryption_padding&oldid=1163481577) is need to ensure IND-CPA security.

In contrast, lattice-based schemes for encryption are inherently randomized due to the problem definition itself. In particular, in the LWE based encryption scheme outlined above, there are two distinct elements of randomness:
- **(1)** The error (or noise) $\varepsilon$ drawn from the distribution $\chi$
- **(2)** The random binary vectors $\mathbf{r} \in \{0,1\}^N$ used for encrypting each bit in the message.

The errors $\varepsilon$ contribute to the security of the public key, ensuring that it's computationally hard to deduce the secret key $\mathbf{s}$. The random binary vectors $\mathbf{r}$ on the other hand provide the essential randomness needed for making repeated encryptions of the same plaintext bit non-deterministic. Thus LWE based schemes are considered IND-CPA secure without the need for external mechanisms such as padding.

Modern cryptosystems aim to achieve so called [**IND-CCA**](https://en.wikipedia.org/w/index.php?title=Ciphertext_indistinguishability&oldid=1130147047) security which stands for **indistinguishability under chosen-ciphertext attack**. In this setting the adversary has the ability to obtain decryptions of a non-trival set of ciphertexts of their choosing with the aim of extracting information to subsequently break the cryptosystem. A scheme is IND-CCA secure if, even with this capability, the adversary cannot do better than random guessing when trying to distinguish encrypted messages. IND-CCA is a stronger security notion than IND-CPA and subsumes it.

Quantum safe KEMs such as **Kyber** are designed to be IND-CCA secure. This is achieved in two steps:
1. An IND-CPA secure public key encryption(PKE) scheme is defined. In the case of Kyber such a PKE is based on **Module-LWE**.
2. A variant of the [Fujisaki-Okamoto transform](https://doi.org/10.1007/s00145-011-9114-1) (FO) is applied to obtain a CCA-secure KEM. The FO transformation is a generic method to convert encryption schemes that are IND-CPA secure into ones that are IND-CCA secure. For details we refer readers to the [original papers](https://doi.org/10.1007/s00145-011-9114-1).

For more information on the security features of **Kyber** and **Dilithium**, as well as reference implementations in various programming languages, you are encouraged to consult the [CRYSTALS suite documentation](https://pq-crystals.org/).

## Hybrid cryptography

Hybrid cryptography refers to using quantum-safe public-key algorithms are combined with traditional public key algorithms (like RSA or elliptic curves) such that the solution is at least no less secure than existing traditional cryptography.

This can help address 2 issues:

1. Existing standards may mandate specific algorithms to be used for encryption, or that such algorithms are in some way approved. To add quantum safety, encryption could first be done using the regular algorithm, but then further protected by a quantum-safe algorithm, whilst still meeting required standards.

2. Quantum algorithms are new, and further analysis is needed to ensure they are indeed safe and correct. For example one of the initial algorithms shortlisted by NIST was subsequently cracked and proven to be insecure. To mitigate this risk, encryption can be done both using a standard, well-reviewed, secure (apart from quantum!) algorithm, in addition to a post-quantum algorithm.

## Summary
The migration to quantum-safe cryptography (QSC) of asymmetric key cryptosystems is necessitated by the expected advent of quantum computers that can break traditional asymmetric key algorithms predicated on the classical hardness of NP-intermediate problems. QSC is conjectured to be robust to quantum attacks because its security is based on NP-hard problems, which generally cannot be efficiently solved even with quantum computers.

In this context, hard problems rooted in the theory of mathematical lattices such as the learning with errors (LWE) problem have emerged as leading contenders for QSC standardization. In particular, the CRYSTALS-Kyber and CRYSTALS-Dilithium algorithms, based on modular lattices, are well positioned as near-term alternatives to popular asymmetric key protocols like RSA, Elliptic Curve, Diffie-Hellman, and DSA.

As we navigate the complex landscape of quantum-safe cryptography, embracing these innovative solutions will be pivotal in upholding the integrity and confidentiality of our digital interactions in the quantum era.